### Leslie Matrix

&nbsp;

The Leslie matrix is a square matrix to model and analyze the dynamics of age-structured populations. It is a projection matrix to study the growth and stability of populations over time.

In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
os.chdir('C:/Users/tm/Downloads/utas/thesis/chapter1/maria')

In [2]:
#number of columns and rows in leslie matrix
maxage=5

### cleanse

In [3]:
#read files
df=pd.ExcelFile('Maria_Island_fundamentals_2012_to_2019_NRETas.xlsx').parse('Sheet0')

In [4]:
#sort by date
grande=df[['INDIVIDUAL','OBSERVATION_DATE','BIRTH_DATE','NUMBER_ACTIVE_TEATS','GENDER']].sort_values(
    ['INDIVIDUAL','OBSERVATION_DATE'])
grande.reset_index(inplace=True,drop=True)

#datetimeindex
grande['OBSERVATION_DATE']=pd.to_datetime(grande['OBSERVATION_DATE'])
grande['BIRTH_DATE']=pd.to_datetime(grande['BIRTH_DATE'])

### estimate transition rate

In [5]:
#compute age
grande['age']=(grande['OBSERVATION_DATE']-grande['BIRTH_DATE']).apply(lambda x:x.days//365)

In [6]:
#for each year,each devil only counts once
grande['year']=grande['OBSERVATION_DATE'].dt.year
agestructure=grande.loc[grande[['year','INDIVIDUAL']].drop_duplicates().index]

In [7]:
#count
agesummary=agestructure.groupby('age').count()[['INDIVIDUAL']]
agesummary.reset_index(inplace=True)

In [8]:
#remove devil older than 10 yr which should be error
agesummary=agesummary[agesummary['age']<=10]

#move anything older than maxage yr into yr maxage category
lastrow=agesummary['INDIVIDUAL'][agesummary['age']>=maxage].sum()
agesummary['INDIVIDUAL'][agesummary['age']==maxage]=lastrow
agesummary=agesummary.iloc[:maxage+1]

In [9]:
#estimate relationship between age and frequency
#plus one to avoid log(0)
agesummary['logage']=np.log(agesummary['age']+1)

m=sm.OLS(agesummary['INDIVIDUAL'],sm.add_constant(agesummary['logage'])).fit()

In [10]:
#use fitted data to compute survivalship
agesummary['smoothed data']=m.predict()
agesummary['survivalship']=agesummary['smoothed data']/agesummary['smoothed data'].iloc[0]

### estimate fecundity

In [11]:
#for each year,each devil only counts once
fecundity=grande.groupby(['year','INDIVIDUAL']).max()[['NUMBER_ACTIVE_TEATS','age','GENDER']]
fecundity.reset_index(inplace=True)

In [12]:
#remove na female
female=fecundity[fecundity['GENDER']=='Female'].copy()
female=female.loc[female['NUMBER_ACTIVE_TEATS'].dropna().index]

#set na male to zero
male=fecundity[fecundity['GENDER']=='Male'].copy()
fecundity=pd.concat([female,male])
fecundity['NUMBER_ACTIVE_TEATS']=fecundity['NUMBER_ACTIVE_TEATS'].fillna(0)

In [13]:
#comput mean fecundity
reproduction=fecundity.groupby('age').mean()
reproduction.reset_index(inplace=True)

C:\Users\tm\AppData\Local\Temp\ipykernel_12504\162338182.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  reproduction=fecundity.groupby('age').mean()


In [14]:
#remove devil older than 10 yr which should be error
reproduction=reproduction[reproduction['age']<=10]

#move anything older than maxage yr into yr maxage category
lastrow=reproduction['NUMBER_ACTIVE_TEATS'][reproduction['age']>=maxage].mean()
reproduction['NUMBER_ACTIVE_TEATS'][reproduction['age']==maxage]=lastrow
reproduction=reproduction.iloc[:maxage+1]

### compute leslie matrix

In [15]:
#generate leslie matrix
lesliematrix=np.zeros((maxage+1,maxage+1))
lesliematrix[0]=reproduction['NUMBER_ACTIVE_TEATS'].tolist()
for i in range(1,maxage+1):
    lesliematrix[i][i-1]=agesummary['survivalship'].iloc[i]

In [16]:
#show
for i in lesliematrix:
    print(i)

[0.         0.2020202  0.54444444 1.19444444 0.82608696 0.58333333]
[0.78169418 0.         0.         0.         0.         0.        ]
[0.         0.65399346 0.         0.         0.         0.        ]
[0.         0.         0.56338836 0.         0.         0.        ]
[0.         0.         0.         0.49310958 0.         0.        ]
[0.         0.         0.         0.         0.43568764 0.        ]


In [17]:
#dominant eigenvalue still smaller than one
#the population eventually will decline
max(np.abs(np.linalg.eigvals(lesliematrix)))

0.9810010800520539